## Assumed to be directed graphs unless stated otherwise

In [1]:
def adjacency_matrix_to_list(adj_matrix):
    adj_list = {}
    num_nodes = len(adj_matrix)

    for i in range(num_nodes):
        adj_list[i] = []
        for j in range(num_nodes):
            if adj_matrix[i][j] != 0:
                adj_list[i].append(j)

    return adj_list

In [2]:
from collections import deque

def dfs(graph):
    visited = set()

    for node in graph:
        if node not in visited:
            explore_recursion(graph, node, visited)            

def explore_recursion(graph, root, visited=None):
    """
    not to be used directly since root node may not reach all other node
    in directed graphs, only source nodes can reach all other nodes 
    if used directly: only applicable to fully connected and undirected graphs.
    """
    
    if visited == None:
        visited = set()
        
    if root in visited:
        return
    
    # do something to root
    print(root)
    visited.add(root)
    
    neighbors = sorted(graph[root])
    for neighbor in neighbors:
        explore_recursion(graph, neighbor, visited)
        
def explore_iterative(graph, root, visited=None):
    dq = deque()
    
    if visited == None:
        visited = set()
        
    dq.append(root)
    while dq:
        node = dq.pop()
        
        if node not in visited:
            
            # do something to node
            print(node)
            visited.add(node)
            
            neighbors = reversed(sorted(graph[node]))
            dq.extend(neighbors)

In [3]:
def dfs_clock(graph):
    
    visited = set()

    clock = [0]
    pre = {v: 0 for v in graph}
    post = {v: 0 for v in graph}
    
    for node in graph:
        if node not in visited:
            explore_clock(graph, node, visited, pre, post, clock)
    
    return pre, post

def explore_clock(graph, root, visited, pre, post, clock):
    
    if root in visited:
        return
    
    # do something to root
    print(root)
    visited.add(root)
    
    previsit(pre, root, clock)

    neighbors = sorted(graph[root])
    for neighbor in neighbors:
        explore_clock(graph, neighbor, visited, pre, post, clock)
        
    postvisit(post, root, clock)
    
def previsit(pre, root, clock):
    pre[root] = clock[0]
    clock[0] += 1
    
def postvisit(post, root, clock):
    post[root] = clock[0]
    clock[0] += 1

In [4]:
def find_backedge(graph):
    pre, post = dfs_clock(graph)
    backedge = []
    # must use post numbers: pre numbers depend on starting vertex
    for node, neighbors in graph.items():
        for neighbor in neighbors:
            if pre[neighbor] < pre[node] and post[node] < post[neighbor]:
                backedge.append((node, neighbor))
    return pre, post, backedge

In [5]:
def reverse_graph(graph):
    r_graph = {v: [] for v in graph}
    for node in graph:
        for neighbor in graph[node]:
            r_graph[neighbor].append(node)
    return r_graph

def find_scc(graph):
    visited = set()
    cc = {}
    cc_count = 0
    
    r_graph = reverse_graph(graph)
    pre, post = dfs_clock(r_graph)
    
    # source has the highest post number in g
    # therefore, sink has the highest post number in rev-g
    top_sorted = sorted(post, key=post.get, reverse=True)
    
    # remove sink scc one by one
    for node in top_sorted:
        if node not in visited:
            clock = [0]
            pre_cc = {}
            post_cc = {}
            
            explore_clock(graph, node, visited, pre_cc, post_cc, clock)
            cc[str(cc_count)] = {i for i in post_cc}
            cc_count += 1

    return cc

In [8]:
graph = {'A': ['B'],
         'B': ['C', 'D', 'E'],
         'C': ['F'],
         'D': [],
         'E': ['B', 'F', 'G'],
         'F': ['C', 'H'],
         'G': ['H', 'J'],
         'H': ['K'],
         'I': ['G'],
         'J': ['I'],
         'K': ['L'],
         'L': ['J']
        }

dfs(graph)

A
B
C
F
H
K
L
J
I
G
D
E


In [9]:
find_backedge(graph)

A
B
C
F
H
K
L
J
I
G
D
E


({'A': 0,
  'B': 1,
  'C': 2,
  'D': 18,
  'E': 20,
  'F': 3,
  'G': 9,
  'H': 4,
  'I': 8,
  'J': 7,
  'K': 5,
  'L': 6},
 {'A': 23,
  'B': 22,
  'C': 17,
  'D': 19,
  'E': 21,
  'F': 16,
  'G': 10,
  'H': 15,
  'I': 11,
  'J': 12,
  'K': 14,
  'L': 13},
 [('E', 'B'), ('F', 'C'), ('G', 'H'), ('G', 'J')])

In [10]:
find_scc(graph)

A
B
E
C
F
D
G
I
J
L
K
H
G
H
K
L
J
I
D
C
F
B
E
A


{'0': {'G', 'H', 'I', 'J', 'K', 'L'},
 '1': {'D'},
 '2': {'C', 'F'},
 '3': {'B', 'E'},
 '4': {'A'}}